In [1]:
from Dataset_Generator import DatasetGenerator
import torch
import torch
import numpy as np
import random

seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU setups

# Optional: Force deterministic behavior on GPU (may impact performance)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [13]:
grid = [[0 for _ in range(20)] for _ in range(20)]  # Generate grid map
dataset_generator = DatasetGenerator(num_cases=100, num_agents=6, grid=grid)
cases = dataset_generator.generate_cases()
# dataset_generator.save_cases_to_file(cases, "dataset.json")
# print(f"Generated and saved {len(cases)} cases.")
# cases = dataset_generator.load_cases_from_file("dataset.json")

case  0
case added
case  1
case added
case  2
case added
case  3
case added
case  4
case added
case  5
case added
case  6
case added
case  7
case added
case  8
case added
case  9
case added
case  10
case added
case  11
case added
case  12
case added
case  13
case added
case  14
case added
case  15
case added
case  16
case added
case  17
case added
case  18
case added
case  19
case added
case  20
case added
case  21
case added
case  22
case added
case  23
case added
case  24
case added
case  25
case added
case  26
case added
case  27
case added
case  28
case added
case  29
case added
case  30
case added
case  31
case added
case  32
case added
case  33
case added
case  34
case added
case  35
case added
case  36
case added
case  37
case added
case  38
case added
case  39
case added
case  40
case added
case  41
case added
case  42
case added
case  43
case added
case  44
case added
case  45
case added
case  46
case added
case  47
case added
case  48
case added
case  49
case added
case  50
c

In [14]:
from Extarct_Actions import Action_Extractor
action_extractor = Action_Extractor(cases, 6)
actions = action_extractor.extract()

In [15]:
print(actions)

{0: {0: [1, 1, 2, 1, 2, 3], 1: [1, 1, 2, 4, 2, 3], 2: [1, 1, 2, 1, 2, 3], 3: [2, 1, 2, 4, 2, 3], 4: [1, 1, 3, 0, 3, 2], 5: [2, 4, 0, 0, 2, 3], 6: [1, 1, 0, 0, 2, 2], 7: [2, 4, 0, 0, 2, 3], 8: [1, 1, 0, 0, 2, 2], 9: [2, 4, 0, 0, 3, 3], 10: [1, 1, 0, 0, 2, 0], 11: [2, 4, 0, 0, 3, 0], 12: [1, 1, 0, 0, 2, 0], 13: [2, 4, 0, 0, 3, 0], 14: [1, 0, 0, 0, 2, 0], 15: [2, 0, 0, 0, 3, 0], 16: [0, 0, 0, 0, 0, 0]}, 1: {0: [2, 4, 1, 1, 2, 1], 1: [2, 4, 1, 1, 2, 1], 2: [2, 4, 1, 1, 1, 4], 3: [2, 4, 1, 1, 2, 1], 4: [2, 4, 1, 1, 1, 1], 5: [2, 4, 1, 1, 2, 2], 6: [2, 4, 1, 1, 1, 1], 7: [2, 4, 1, 4, 2, 2], 8: [1, 3, 1, 1, 1, 1], 9: [2, 4, 1, 4, 2, 2], 10: [1, 3, 1, 1, 1, 1], 11: [2, 4, 1, 4, 2, 2], 12: [1, 3, 4, 1, 0, 1], 13: [2, 4, 1, 4, 0, 2], 14: [1, 3, 4, 1, 0, 1], 15: [2, 4, 1, 4, 0, 2], 16: [0, 3, 4, 1, 0, 0], 17: [0, 4, 1, 4, 0, 0], 18: [0, 3, 4, 1, 0, 0], 19: [0, 4, 1, 4, 0, 0], 20: [0, 3, 4, 1, 0, 0], 21: [0, 4, 0, 4, 0, 0], 22: [0, 3, 0, 0, 0, 0], 23: [0, 0, 0, 0, 0, 0]}, 2: {0: [4, 1, 4, 3, 4, 2]

In [5]:
actions = torch.tensor([actions[key][subkey] for key in actions for subkey in actions[key]])

In [6]:
print(actions)

tensor([[1, 1, 2, 1, 3, 2],
        [1, 1, 2, 1, 3, 2],
        [2, 0, 2, 1, 3, 2],
        [1, 0, 2, 0, 4, 2],
        [2, 0, 2, 0, 3, 2],
        [0, 0, 2, 0, 0, 2],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]])


In [25]:
num_of_robots = 6
from pre_processing import Preprocessing
from torch.utils.data import DataLoader, random_split
from Encoder import ObservationEncoder
cnn = ObservationEncoder()
p = Preprocessing(grid,cases,3)
data_tensors  = p.begin()
def prepare_data(tensors):
        dataset = []
        action_extractor = Action_Extractor(cases, num_of_robots)
        actions = action_extractor.extract()
        for itr in range(len(tensors.keys())):
            # first_channels = self.tensors[itr]['channel 1']
            second_channels = tensors[itr]['channel 2']
            third_channels = tensors[itr]['channel 3']
            
            for step in range(len(second_channels)):
                batch_channels = []
                for i in range(num_of_robots):
                    agent_channels = np.stack([
                        second_channels[step][i],
                        third_channels[step][i]
                    ])
                    batch_channels.append(agent_channels)
                batch_tensor = torch.tensor(np.array(batch_channels), dtype=torch.float32)
                robots_actions = torch.tensor(np.array(actions[itr][step]),dtype=torch.float32)
                dataset.append((batch_tensor,robots_actions))
        return dataset
dataset = prepare_data(data_tensors)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
v = 0
for tensor, act in train_loader:
    if v ==0:
        print(tensor.shape)
        batch_size, num_agents = tensor.shape[0], tensor.shape[1]
        print(batch_size)
        print(num_agents)
        print(tensor.shape[2])
        tensor = tensor.view(batch_size * num_agents,tensor.shape[2],tensor.shape[3],tensor.shape[4])
        print(tensor.shape)
        cnn_output = cnn(tensor)
        print(cnn_output)
        v = 1
        break

torch.Size([64, 6, 2, 9, 9])
64
6
2
torch.Size([384, 2, 9, 9])
tensor([[-0.1989, -0.3127,  0.0088,  ...,  0.2004,  0.8575, -0.2141],
        [-0.3588,  0.2542, -0.0760,  ..., -0.3892,  0.9921, -0.1520],
        [-0.0742, -0.2151,  0.3769,  ..., -0.2701,  0.3254,  0.2294],
        ...,
        [-0.2631, -0.4102,  0.1386,  ..., -0.2854,  0.6867, -0.1018],
        [-0.4848,  0.6751,  0.1950,  ..., -0.5596,  0.6052, -0.4705],
        [-0.2631, -0.4102,  0.1386,  ..., -0.2854,  0.6867, -0.1018]],
       grad_fn=<AddmmBackward0>)


In [3]:
from pre_processing import Preprocessing
# p = Preprocessing(grid,cases,3)
# data_tensors  = p.begin()

In [4]:
from Encoder import Encode
# encoder = Encode(data_tensors,6)
# encoded_tensors = encoder.begin()

In [5]:
from Adjacency_Matrix import adj_mat

In [6]:
# adj = adj_mat(cases,3)
# adj_matrices = adj.get_adj_mat()

In [7]:
from GNN_file import Communication_GNN

c:\Users\moham\anaconda3\Lib\site-packages\torch_geometric\typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: [WinError 127] The specified procedure could not be found
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "


In [8]:
# comm_gnn = Communication_GNN(encoded=encoded_tensors, adj_mat=adj_matrices, num_of_agents=6)
# gnn_features = comm_gnn.begin()

In [9]:
from Extarct_Actions import Action_Extractor

In [10]:
# action_extractor = Action_Extractor(cases, 6)
# actions = action_extractor.extract()

In [11]:
# gnn_features = torch.cat([gnn_features[key][subkey] for key in gnn_features for subkey in gnn_features[key]])

In [12]:
# actions = torch.tensor([actions[key][subkey] for key in actions for subkey in actions[key]])

In [13]:
# reshaped_gnn_features = []
# temp = []
# c = 1
# for i in range(len(gnn_features)):
#     temp.append(gnn_features[i].tolist())
#     if c % 6 == 0:
#         reshaped_gnn_features.append(torch.Tensor(temp))
#         temp = []
#     c+=1

In [14]:
# dataset = []
# for i in range(len(actions)):
#     dataset.append((reshaped_gnn_features[i],actions[i]))

In [1]:
from MLP_Action import ActionMLP
mlp = ActionMLP(input_dim= 128, hidden_dim= 128, num_actions= 5)

In [16]:
# from Model_Generation import Generate_Model
# model = Generate_Model(model = mlp, dataset=dataset,num_epochs=150)

In [17]:
# model.train_model()

In [18]:
# model.test_model()

In [ ]:
new_model = ActionMLP(input_dim= 128, hidden_dim= 128, num_actions= 5)
state_dict = torch.load('trained_model_6.pth')
new_model.load_state_dict(state_dict)
new_model.eval()  # set the model to evaluation mode

ActionMLP(
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=128, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [20]:
num_of_robots = 6
new_dataset_generator = DatasetGenerator(num_cases=100, num_agents=num_of_robots, grid=grid)
eval_cases = new_dataset_generator.generate_cases()

case  0
case added
case  1
case added
case  2
case added
case  3
case added
case  4
case added
case  5
case added
case  6
case added
case  7
case added
case  8
case added
case  9
case added
case  10
case added
case  11
case added
case  12
case added
case  13
case added
case  14
case added
case  15
case added
case  16
case added
case  17
case added
case  18
case added
case  19
case added
case  20
case added
case  21
case added
case  22
case added
case  23
case added
case  24
case added
case  25
case added
case  26
case added
case  27
case added
case  28
case added
case  29
case added
case  30
case added
case  31
case added
case  32
case added
case  33
case added
case  34
case added
case  35
case added
case  36
case added
case  37
case added
case  38
case added
case  39
case added
case  40
case added
case  41
case added
case  42
case added
case  43
case added
case  44
case added
case  45
case added
case  46
case added
case  47
case added
case  48
case added
case  49
case added
case  50
c

In [21]:
print(eval_cases[0]['start_positions'])
print(eval_cases[0]['goal_positions'])
print(eval_cases[0]['paths'])
# action_extractor = Action_Extractor(eval_cases, 6)
# actions = action_extractor.extract()
# print(actions)

[(3, 0), (8, 7), (7, 4), (3, 17), (2, 18), (13, 1)]
[(0, 2), (6, 7), (16, 19), (0, 17), (6, 17), (13, 7)]
[[(3, 0), (2, 0), (1, 0), (1, 1), (0, 1), (0, 2)], [(8, 7), (7, 7), (6, 7)], [(7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (8, 11), (8, 12), (9, 12), (9, 13), (10, 13), (10, 14), (11, 14), (11, 15), (12, 15), (12, 16), (13, 16), (13, 17), (14, 17), (14, 18), (15, 18), (15, 19), (16, 19)], [(3, 17), (2, 17), (1, 17), (0, 17)], [(2, 18), (3, 18), (4, 18), (5, 18), (5, 17), (6, 17)], [(13, 1), (13, 2), (13, 3), (13, 4), (13, 5), (13, 6), (13, 7)]]


In [22]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
success = 0
actions = [[0,0],[-1,0],[0,1],[1,0],[0,-1]]
seed = 42
torch.manual_seed(seed)
with torch.inference_mode():
    for case in eval_cases:
        threshold = 0
        for path in case['paths']:
            threshold+= len(path)
        threshold = 3*threshold # from the paper
        num_of_steps = 0
        robots_reached = set()
        step_number = 0
        current_nodes = []
        path_robot_zero = []
        while num_of_steps <= threshold and len(robots_reached) <=6:
            paths = []
            goals = []
            for robot in range(num_of_robots):
                if step_number == 0:
                    current_node = case['paths'][robot][step_number]
                    current_nodes.append(list(current_node))
                    if robot == 0:
                        print(current_node)
                        print(list(case['paths'][robot][-1]))
                    paths.append([list(current_node)])
                    goals.append(list(case['goal_positions'][robot]))
                else:
                    paths.append([current_nodes[robot]])
                    goals.append(list(case['goal_positions'][robot]))
            # print(current_nodes[0])
            step_case = {"start_positions": current_nodes, "goal_positions": goals,"paths":paths}
            # print(paths)
            # print("start_preprocessing")

            p_new = Preprocessing(grid,[step_case],3)
            data_tensors_new  = p_new.begin()
            # data_tensors_new[0]['channel 3'][0] = np.array([
            #     [
            #         [0 if (ii != 4 or jj != 4) else 1 for ii in range(9)]
            #         for jj in range(9)
            #     ]
            #         for kk in range(num_of_robots)
            #     ])

            # if num_of_steps ==26:
            #     # print(len(step_case['start_positions']))
            #     print(step_case)
            #     print(current_nodes[3])
            #     # print(np.rot90(data_tensors_new[0]['channel 2'][0][1].T))
            #     print(data_tensors_new[0]['channel 2'][0][3])

            # print("start_encoding")
            # torch.manual_seed(42)
            new_encoder = Encode(data_tensors_new,num_of_robots)
            new_encoded_tensors = new_encoder.begin()
            # print(new_encoded_tensors[0])
            # print("start adj")
            new_adj = adj_mat([step_case],3)
            new_adj_matrices = new_adj.get_adj_mat()
            # print("start gnn")
            new_comm_gnn = Communication_GNN(encoded=new_encoded_tensors, adj_mat=new_adj_matrices, num_of_agents=num_of_robots)
            new_gnn_features = new_comm_gnn.begin()
            gnn_features_cat = torch.cat([new_gnn_features[key][subkey] for key in new_gnn_features for subkey in new_gnn_features[key]])
            reshaped_gnn_features = []
            temp = []
            c = 1
            for i in range(len(gnn_features_cat)):
                temp.append(gnn_features_cat[i].tolist())
                if c % num_of_robots == 0:
                    reshaped_gnn_features.append(torch.Tensor(temp))
                    temp = []
                c+=1
            # print(reshaped_gnn_features[0][0])
            # print("start check")
            future_nodes = list(range(0,num_of_robots))
            for robot in range(num_of_robots):
                if robot not in robots_reached:
                    # prediction = model.model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))
                    prediction = new_model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))
                    # print(prediction)
                    predicted_class = torch.argmax(prediction, dim=1)
                    right_state = False
                    #--------------------------------------------------------------------------------------------------
                    # if robot == 1 and robot not in robots_reached:
                        # print(current_nodes[robot])
                        # print(predicted_class)
                    #--------------------------------------------------------------------------------------------------
                    # if (current_nodes[robot][0]+1 == goals[robot][0]) and (current_nodes[robot][1] == goals[robot][1]):
                    #     predicted_class = 3
                    #     right_state = True
                    #--------------------------------------------------------------------------------------------------
                    # path_robot_zero.append(predicted_class)
                    #--------------------------------------------------------------------------------------------------
                    next = [current_nodes[robot][0] + actions[predicted_class][0],current_nodes[robot][1] + actions[predicted_class][1]]
                    # future_nodes[robot] = (next[0],next[1])
                    if (next[0]>=20 or next[0]<0 or next[1]>=20 or next[1]<0):
                        # current_nodes[robot] = [current_nodes[robot][0] + actions[0][0],current_nodes[robot][1] + actions[0][1]]
                        pass
                    else:
                        current_nodes[robot] = [current_nodes[robot][0] + actions[predicted_class][0],current_nodes[robot][1] + actions[predicted_class][1]]
                    if (current_nodes[robot][0] == goals[robot][0]) and (current_nodes[robot][1] == goals[robot][1]):
                        robots_reached.add(robot)
                        print("trueeeeee")
            step_number = 1
            num_of_steps+=1
        if len(robots_reached) == num_of_robots:
            success += 1


(3, 0)
[0, 2]


C:\Users\moham\AppData\Local\Temp\ipykernel_19244\3611235621.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prediction = new_model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))


trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(14, 18)
[2, 12]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(2, 17)
[7, 3]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(19, 5)
[10, 12]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(8, 4)
[4, 16]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(12, 19)
[9, 13]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(19, 6)
[3, 11]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(4, 4)
[9, 12]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(18, 17)
[2, 16]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(15, 13)
[1, 12]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(8, 14)
[5, 13]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(14, 9)
[1, 10]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(5, 2)
[13, 18]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(9, 14)
[16, 8]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
trueeeeee


In [23]:
print(success)

84


In [24]:
print(robots_reached)

{0, 1, 2, 3, 4, 5}


In [25]:
num_of_robots = 4
new_dataset_generator = DatasetGenerator(num_cases=100, num_agents=num_of_robots, grid=grid)
eval_cases = new_dataset_generator.generate_cases()

case  0
case added
case  1
case added
case  2
case added
case  3
case added
case  4
case added
case  5
case added
case  6
case added
case  7
case added
case  8
case added
case  9
case added
case  10
case added
case  11
case added
case  12
case added
case  13
case added
case  14
case added
case  15
case added
case  16
case added
case  17
case added
case  18
case added
case  19
case added
case  20
case added
case  21
case added
case  22
case added
case  23
case added
case  24
case added
case  25
case added
case  26
case added
case  27
case added
case  28
case added
case  29
case added
case  30
case added
case  31
case added
case  32
case added
case  33
case added
case  34
case added
case  35
case added
case  36
case added
case  37
case added
case  38
case added
case  39
case added
case  40
case added
case  41
case added
case  42
case added
case  43
case added
case  44
case added
case  45
case added
case  46
case added
case  47
case added
case  48
case added
case  49
case added
case  50
c

In [ ]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
success = 0
actions = [[0,0],[-1,0],[0,1],[1,0],[0,-1]]
seed = 42
torch.manual_seed(seed)
with torch.inference_mode():
    for case in eval_cases:
        threshold = 0
        for path in case['paths']:
            threshold+= len(path)
        threshold = 3*threshold # from the paper
        num_of_steps = 0
        robots_reached = set()
        step_number = 0
        current_nodes = []
        path_robot_zero = []
        while num_of_steps <= threshold and len(robots_reached) <=6:
            paths = []
            goals = []
            for robot in range(num_of_robots):
                if step_number == 0:
                    current_node = case['paths'][robot][step_number]
                    current_nodes.append(list(current_node))
                    if robot == 0:
                        print(current_node)
                        print(list(case['paths'][robot][-1]))
                    paths.append([list(current_node)])
                    goals.append(list(case['goal_positions'][robot]))
                else:
                    paths.append([current_nodes[robot]])
                    goals.append(list(case['goal_positions'][robot]))
            # print(current_nodes[0])
            step_case = {"start_positions": current_nodes, "goal_positions": goals,"paths":paths}
            # print(paths)
            # print("start_preprocessing")

            p_new = Preprocessing(grid,[step_case],3)
            data_tensors_new  = p_new.begin()
            # data_tensors_new[0]['channel 3'][0] = np.array([
            #     [
            #         [0 if (ii != 4 or jj != 4) else 1 for ii in range(9)]
            #         for jj in range(9)
            #     ]
            #         for kk in range(num_of_robots)
            #     ])

            # if num_of_steps ==26:
            #     # print(len(step_case['start_positions']))
            #     print(step_case)
            #     print(current_nodes[3])
            #     # print(np.rot90(data_tensors_new[0]['channel 2'][0][1].T))
            #     print(data_tensors_new[0]['channel 2'][0][3])

            # print("start_encoding")
            # torch.manual_seed(42)
            new_encoder = Encode(data_tensors_new,num_of_robots)
            new_encoded_tensors = new_encoder.begin()
            # print(new_encoded_tensors[0])
            # print("start adj")
            new_adj = adj_mat([step_case],3)
            new_adj_matrices = new_adj.get_adj_mat()
            # print("start gnn")
            new_comm_gnn = Communication_GNN(encoded=new_encoded_tensors, adj_mat=new_adj_matrices, num_of_agents=num_of_robots)
            new_gnn_features = new_comm_gnn.begin()
            gnn_features_cat = torch.cat([new_gnn_features[key][subkey] for key in new_gnn_features for subkey in new_gnn_features[key]])
            reshaped_gnn_features = []
            temp = []
            c = 1
            for i in range(len(gnn_features_cat)):
                temp.append(gnn_features_cat[i].tolist())
                if c % num_of_robots == 0:
                    reshaped_gnn_features.append(torch.Tensor(temp))
                    temp = []
                c+=1
            # print(reshaped_gnn_features[0][0])
            # print("start check")
         
            for robot in range(num_of_robots):
                if robot not in robots_reached:
                    # prediction = model.model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))
                    prediction = new_model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))
                    # print(prediction)
                    predicted_class = torch.argmax(prediction, dim=1)
                    #--------------------------------------------------------------------------------------------------
                    # if robot == 1 and robot not in robots_reached:
                        # print(current_nodes[robot])
                        # print(predicted_class)
                    #--------------------------------------------------------------------------------------------------
                    # if (current_nodes[robot][0]+1 == goals[robot][0]) and (current_nodes[robot][1] == goals[robot][1]):
                    #     predicted_class = 3
                    #     right_state = True
                    #--------------------------------------------------------------------------------------------------
                    # path_robot_zero.append(predicted_class)
                    #--------------------------------------------------------------------------------------------------
                    next = [current_nodes[robot][0] + actions[predicted_class][0],current_nodes[robot][1] + actions[predicted_class][1]]
                    # future_nodes[robot] = (next[0],next[1])
                    if (next[0]>=20 or next[0]<0 or next[1]>=20 or next[1]<0):
                        # current_nodes[robot] = [current_nodes[robot][0] + actions[0][0],current_nodes[robot][1] + actions[0][1]]
                        pass
                    else:
                        current_nodes[robot] = [current_nodes[robot][0] + actions[predicted_class][0],current_nodes[robot][1] + actions[predicted_class][1]]
                    if (current_nodes[robot][0] == goals[robot][0]) and (current_nodes[robot][1] == goals[robot][1]):
                        robots_reached.add(robot)
                        print("trueeeeee")
            step_number = 1
            num_of_steps+=1
        if len(robots_reached) == num_of_robots:
            success += 1


(0, 9)
[7, 14]


C:\Users\moham\AppData\Local\Temp\ipykernel_19244\3611235621.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prediction = new_model(torch.tensor(reshaped_gnn_features[0][robot],dtype=torch.float32).unsqueeze(0).to(device))


trueeeeee
trueeeeee
trueeeeee
trueeeeee
(13, 19)
[1, 14]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(7, 17)
[18, 10]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(16, 7)
[5, 13]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(10, 13)
[15, 7]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(13, 13)
[6, 10]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(1, 8)
[18, 17]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(6, 6)
[7, 17]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(9, 8)
[10, 6]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(4, 0)
[4, 6]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(17, 10)
[5, 4]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(1, 2)
[4, 0]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(19, 15)
[0, 0]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(13, 5)
[19, 16]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(12, 14)
[2, 12]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(5, 2)
[1, 7]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(8, 17)
[18, 4]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(18, 12)
[11, 1]
trueeeeee
trueeeeee
trueeeeee
trueeeeee
(19, 19)


In [27]:
print(success)

97


In [28]:
print(robots_reached)

{0, 1, 2, 3}
